<a href="https://colab.research.google.com/github/Ha1ion/2025_NLP_HW2/blob/main/nlp_hw2_LSTM_20_wrong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM-arithmetic

## Dataset
- [Arithmetic dataset](https://drive.google.com/file/d/1cMuL3hF9jefka9RyF4gEBIGGeFGZYHE-/view?usp=sharing)

此作業有使用Gemini幫忙下註解

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install seaborn
! pip install opencc
! pip install -U scikit-learn

import numpy as np
import pandas as pd
import torch
import torch.nn
import torch.nn.utils.rnn
import torch.utils.data
import matplotlib.pyplot as plt
import seaborn as sns
import opencc
import os
from sklearn.model_selection import train_test_split

data_path = './data'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 70.8 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1


In [4]:
data_path = '/content/drive/MyDrive/2025_NLP_HW2'
df_train = pd.read_csv(os.path.join(data_path, 'arithmetic_train_noisy.csv'))
df_eval = pd.read_csv(os.path.join(data_path, 'arithmetic_eval.csv'))
df_train.head()

,Unnamed: 0,src,tgt
0,2285313,14*(43+20)=,882
1,317061,(6+1)*5=,31
2,718770,13+32+29=,74
3,170195,31*(3-11)=,-248
4,2581417,24*49+1=,1177


實驗二：訓練三位數，評估二位數

In [ ]:
import random
import pandas as pd

def generate_arithmetic_data(num_samples, num_digits):
    data = []
    low = 10**(num_digits - 1)
    high = 10**num_digits - 1
    operators = ['+', '-', '*']

    for _ in range(num_samples):
        n1 = random.randint(low, high)
        n2 = random.randint(low, high)
        op = random.choice(operators)

        # 避免過於複雜的表達式，這裡只用兩個數字
        expression = f"{n1}{op}{n2}"

        try:
            # 使用 eval() 來安全計算結果
            answer = eval(expression)
            src = f"{expression}="
            tgt = str(answer)
            data.append({'src': src, 'tgt': tgt})
        except:
            continue

    return pd.DataFrame(data)

# 生成 26 萬筆三位數的訓練資料 (數量不用太多，足以訓練出模式即可)
print("Generating 3-digit training data...")
df_train_3digit = generate_arithmetic_data(260000, 3)
df_train_3digit.to_csv('arithmetic_train_3digit.csv', index=False)
print("File 'arithmetic_train_3digit.csv' has been generated.")

Generating 3-digit training data...
File 'arithmetic_train_3digit.csv' has been generated.


In [ ]:
# transform the input data to string
df_train['tgt'] = df_train['tgt'].apply(lambda x: str(x))
df_train['src'] = df_train['src'].add(df_train['tgt'])
df_train['len'] = df_train['src'].apply(lambda x: len(x))

df_eval['tgt'] = df_eval['tgt'].apply(lambda x: str(x))

20% wrong

In [ ]:
import pandas as pd
import numpy as np
import os

# --- 1. 設定您存放資料的路徑 ---
data_path = '/content/drive/MyDrive/2025_NLP_HW2'

# --- 2. 組合出完整的讀取與儲存路徑 ---
original_train_file = os.path.join(data_path, 'arithmetic_train.csv')
output_noisy_file = os.path.join(data_path, 'arithmetic_train_noisy.csv')

try:
    # --- 3. 讀取原始檔案 ---
    print(f"正在從 '{original_train_file}' 讀取資料...")
    df_train = pd.read_csv(original_train_file)
    print("讀取完成。")

    df_train_noisy = df_train.copy()

    # --- 4. 製造並寫入錯誤資料 (高效的向量化版本) ---
    print("正在生成 20% 的錯誤答案 (優化版)...")

    # 隨機選出 20% 的樣本索引
    noisy_indices = df_train_noisy.sample(frac=0.20, random_state=42).index

    # 將 'tgt' 欄位轉換為數值型別以進行計算
    original_answers = pd.to_numeric(df_train_noisy.loc[noisy_indices, 'tgt'], errors='coerce')

    # 過濾掉無法轉換的非數值資料
    valid_indices = original_answers.dropna().index

    # 一次性生成所有隨機偏移量
    num_to_modify = len(valid_indices)
    offsets = np.random.randint(1, 21, size=num_to_modify) * np.random.choice([-1, 1], size=num_to_modify)

    # 進行向量化計算，產生所有錯誤答案
    noisy_answers = original_answers.loc[valid_indices] + offsets

    # 將新的錯誤答案 (轉回字串格式) 一次性地寫回 DataFrame
    df_train_noisy.loc[valid_indices, 'tgt'] = noisy_answers.astype(str)

    print("錯誤答案生成完畢。")

    # 將包含錯誤答案的 DataFrame 儲存為新的 CSV 檔案
    print(f"正在將結果儲存至 '{output_noisy_file}'...")
    df_train_noisy.to_csv(output_noisy_file, index=False)

    print("\n" + "="*50)
    print(f"成功！新的訓練集檔案 'arithmetic_train_noisy.csv' 已經生成。")
    print("="*50)

except FileNotFoundError:
    print(f"錯誤：在 '{data_path}' 中找不到 'arithmetic_train.csv' 檔案。")
except Exception as e:
    print(f"程式執行時發生未預期的錯誤: {e}")

正在從 '/content/drive/MyDrive/2025_NLP_HW2/arithmetic_train.csv' 讀取資料...
讀取完成。
正在生成 20% 的錯誤答案 (優化版)...


/tmp/ipython-input-2944731329.py:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['-1' '-122' '-120' ... '2' '1272' '-11']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_train_noisy.loc[valid_indices, 'tgt'] = noisy_answers.astype(str)


錯誤答案生成完畢。
正在將結果儲存至 '/content/drive/MyDrive/2025_NLP_HW2/arithmetic_train_noisy.csv'...

成功！新的訓練集檔案 'arithmetic_train_noisy.csv' 已經生成。


In [5]:
import pandas as pd
import os
from google.colab import drive

# --- 步驟一：掛載 Google Drive ---
try:
    print("正在掛載 Google Drive...")
    # force_remount=True 確保每次都重新認證，避免連線問題
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive 掛載成功！")

    # --- 步驟二：執行驗證腳本 ---
    # 設定檔案路徑
    data_path = '/content/drive/MyDrive/2025_NLP_HW2'
    original_file = os.path.join(data_path, 'arithmetic_train.csv')
    noisy_file = os.path.join(data_path, 'arithmetic_train_noisy.csv')

    # 檢查檔案是否存在
    if not os.path.exists(original_file):
        raise FileNotFoundError(original_file)
    if not os.path.exists(noisy_file):
        raise FileNotFoundError(noisy_file)

    # 讀取檔案
    print(f"\n正在讀取原始檔案: {original_file}")
    df_original = pd.read_csv(original_file)

    print(f"正在讀取含錯誤答案的檔案: {noisy_file}")
    df_noisy = pd.read_csv(noisy_file)

    # 進行比較與計算
    total_rows = len(df_original)
    print(f"\n檔案總筆數: {total_rows}")

    df_original['tgt'] = df_original['tgt'].astype(str)
    df_noisy['tgt'] = df_noisy['tgt'].astype(str)

    differences = (df_original['tgt'] != df_noisy['tgt']).sum()
    print(f"找到答案不同的筆數: {differences}")

    percent_noisy = (differences / total_rows) * 100

    print("\n" + "="*50)
    print(f"驗證結果：錯誤答案的實際百分比為: {percent_noisy:.2f}%")
    print("="*50)

except FileNotFoundError as e:
    print(f"\n錯誤：找不到檔案 '{e}'。")
    print("請使用 Colab 左側的檔案總管，親自確認該檔案是否存在於您的雲端硬碟路徑中。")
except Exception as e:
    print(f"\n發生未預期的錯誤: {e}")

正在掛載 Google Drive...
Mounted at /content/drive
Google Drive 掛載成功！

正在讀取原始檔案: /content/drive/MyDrive/2025_NLP_HW2/arithmetic_train.csv
正在讀取含錯誤答案的檔案: /content/drive/MyDrive/2025_NLP_HW2/arithmetic_train_noisy.csv

檔案總筆數: 2369250
找到答案不同的筆數: 473850

驗證結果：錯誤答案的實際百分比為: 20.00%


# Build Dictionary
 - The model cannot perform calculations directly with plain text.
 - Convert all text (numbers/symbols) into numerical representations.
 - Special tokens
    - '&lt;pad&gt;'
        - Each sentence within a batch may have different lengths.
        - The length is padded with '&lt;pad&gt;' to match the longest sentence in the batch.
    - '&lt;eos&gt;'
        - Specifies the end of the generated sequence.
        - Without '&lt;eos&gt;', the model will not know when to stop generating.

In [ ]:
char_to_id = {}
id_to_char = {}

# write your code here
# Build a dictionary and give every token in the train dataset an id
# The dictionary should contain <eos> and <pad>
# char_to_id is to conver charactors to ids, while id_to_char is the opposite

# 建立一個包含所有字元的集合
vocab = set()
for text in df_train['src']:
    vocab.update(list(text))

# 特殊 tokens
special_tokens = ['<pad>', '<eos>']

# 建立 char_to_id 和 id_to_char 字典
# <pad> 的 id 必須是 0，因為 PyTorch 的 padding 函式預設用 0 來填充
char_to_id = {token: i for i, token in enumerate(special_tokens + sorted(list(vocab)))}
id_to_char = {i: token for token, i in char_to_id.items()}

# 更新 df_train 中的 'src' 欄位，確保它包含了答案部分
# 這一行在助教的程式碼中是下一個 cell，但邏輯上屬於這裡的前處理
# df_train['src'] = df_train['src'].add(df_train['tgt'])
# 檢查一下助教的程式碼，如果這行已經在下一個 cell，就不用重複加

vocab_size = len(char_to_id)
print('Vocab size{}'.format(vocab_size))

Vocab size18


# Data Preprocessing
 - The data is processed into the format required for the model's input and output. (End with \<eos\> token)


In [ ]:
# Write your code here

def text_to_ids(text, char_map):
    return [char_map[char] for char in text]

# 確保 'src' 包含題目與答案
df_train['src'] = df_train['src'].str.replace('=', '') + '=' + df_train['tgt']

# 建立 char_id_list
df_train['char_id_list'] = df_train['src'].apply(lambda x: text_to_ids(x, char_to_id) + [char_to_id['<eos>']])

# 建立 label_id_list
def create_label(row):
    # 找到等號 '=' 的位置
    equal_sign_index = row['src'].find('=')

    # 題目部分的長度 (包含等號)
    question_len = equal_sign_index + 1

    # 答案部分的字串
    answer_text = row['tgt']

    # 建立 label list，題目部分用 <pad> 填充
    label = [char_to_id['<pad>']] * question_len

    # 加上答案的 id list
    label.extend(text_to_ids(answer_text, char_to_id))

    # 最後加上 <eos>
    label.append(char_to_id['<eos>'])

    return label

df_train['label_id_list'] = df_train.apply(create_label, axis=1)

# 同樣地，為 df_eval 準備 'src' 欄位，後面評估會用到
df_eval['src'] = df_eval['src'].str.replace('=', '') + '='

df_train.head()

,Unnamed: 0,src,tgt,len,char_id_list,label_id_list
0,2285313,14*(43+20)882=882,882,14,"[8, 11, 4, 2, 11, 10, 5, 9, 7, 3, 15, 15, 9, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15,..."
1,317061,(6+1)*535=35,35,10,"[2, 13, 5, 8, 3, 4, 12, 10, 12, 17, 10, 12, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 12, 1]"
2,718770,13+32+2974=74,74,11,"[8, 10, 5, 10, 9, 5, 9, 16, 14, 11, 17, 14, 11...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14, 11, 1]"
3,170195,31*(3-11)-248=-248,-248,14,"[10, 8, 4, 2, 10, 6, 8, 8, 3, 6, 9, 11, 15, 17...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, ..."
4,2581417,24*49+11177=1177,1177,12,"[9, 11, 4, 11, 16, 5, 8, 8, 8, 14, 14, 17, 8, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 8, 14,..."


# Hyper Parameters

|Hyperparameter|Meaning|Value|
|-|-|-|
|`batch_size`|Number of data samples in a single batch|64|
|`epochs`|Total number of epochs to train|10|
|`embed_dim`|Dimension of the word embeddings|256|
|`hidden_dim`|Dimension of the hidden state in each timestep of the LSTM|256|
|`lr`|Learning Rate|0.001|
|`grad_clip`|To prevent gradient explosion in RNNs, restrict the gradient range|1|

In [ ]:
batch_size = 64
epochs = 2
embed_dim = 256
hidden_dim = 256
lr = 0.001
grad_clip = 1

# Data Batching
- Use `torch.utils.data.Dataset` to create a data generation tool called  `dataset`.
- The, use `torch.utils.data.DataLoader` to randomly sample from the `dataset` and group the samples into batches.

- Example: 1+2-3=0
    - Model input: 1 + 2 - 3 = 0
    - Model output: / / / / / 0 &lt;eos&gt;  (the '/' can be replaced with &lt;pad&gt;)
    - The key for the model's output is that the model does not need to predict the next character of the previous part. What matters is that once the model sees '=', it should start generating the answer, which is '0'. After generating the answer, it should also generate&lt;eos&gt;

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, sequences):
        self.sequences = sequences

    def __len__(self):
        # return the amount of data
        return len(self.sequences) # Write your code here

    def __getitem__(self, index):
        # Extract the input data x and the ground truth y from the data
        row = self.sequences.iloc[index]
        x = row['char_id_list'] # Write your code here
        y = row['label_id_list'] # Write your code here
        return x, y

# collate function, used to build dataloader
def collate_fn(batch):
    batch_x = [torch.tensor(data[0]) for data in batch]
    batch_y = [torch.tensor(data[1]) for data in batch]
    batch_x_lens = torch.LongTensor([len(x) for x in batch_x])
    batch_y_lens = torch.LongTensor([len(y) for y in batch_y])

    # Pad the input sequence
    pad_batch_x = torch.nn.utils.rnn.pad_sequence(batch_x,
                                                  batch_first=True,
                                                  padding_value=char_to_id['<pad>'])

    pad_batch_y = torch.nn.utils.rnn.pad_sequence(batch_y,
                                                  batch_first=True,
                                                  padding_value=char_to_id['<pad>'])

    return pad_batch_x, pad_batch_y, batch_x_lens, batch_y_lens

In [ ]:
ds_train = Dataset(df_train[['char_id_list', 'label_id_list']])

In [ ]:
# Build dataloader of train set and eval set, collate_fn is the collate function
ds_train = Dataset(df_train) # Write your code here

dl_train = torch.utils.data.DataLoader(ds_train,
                                       batch_size=batch_size,
                                       shuffle=True,
                                       collate_fn=collate_fn)

# Model Design

## Execution Flow
1. Convert all characters in the sentence into embeddings.
2. Pass the embeddings through an LSTM sequentially.
3. The output of the LSTM is passed into another LSTM, and additional layers can be added.
4. The output from all time steps of the final LSTM is passed through a Fully Connected layer.
5. The character corresponding to the maximum value across all output dimensions is selected as the next character.

## Loss Function
Since this is a classification task, Cross Entropy is used as the loss function.

## Gradient Update
Adam algorithm is used for gradient updates.

In [ ]:
class CharRNN(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(CharRNN, self).__init__()

        self.embedding = torch.nn.Embedding(num_embeddings=vocab_size,
                                            embedding_dim=embed_dim,
                                            padding_idx=char_to_id['<pad>'])

        self.rnn_layer1 = torch.nn.LSTM(input_size=embed_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)

        self.rnn_layer2 = torch.nn.LSTM(input_size=hidden_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)

        self.linear = torch.nn.Sequential(torch.nn.Linear(in_features=hidden_dim,
                                                          out_features=hidden_dim),
                                          torch.nn.ReLU(),
                                          torch.nn.Linear(in_features=hidden_dim,
                                                          out_features=vocab_size))

    def forward(self, batch_x, batch_x_lens):
        return self.encoder(batch_x, batch_x_lens)

    # The forward pass of the model
    def encoder(self, batch_x, batch_x_lens):
        batch_x = self.embedding(batch_x)

        batch_x = torch.nn.utils.rnn.pack_padded_sequence(batch_x,
                                                          batch_x_lens,
                                                          batch_first=True,
                                                          enforce_sorted=False)

        batch_x, _ = self.rnn_layer1(batch_x)
        batch_x, _ = self.rnn_layer2(batch_x)

        batch_x, _ = torch.nn.utils.rnn.pad_packed_sequence(batch_x,
                                                            batch_first=True)

        batch_x = self.linear(batch_x)

        return batch_x

    def generator(self, start_char, max_len=200):

        char_list = [char_to_id[c] for c in start_char]

        next_char = None

        while len(char_list) < max_len:
            # Write your code here
            # Pack the char_list to tensor
            # Input the tensor to the embedding layer, LSTM layers, linear respectively

            input_seq = torch.LongTensor([char_list]).to(next(self.parameters()).device)
            y = self.encoder(input_seq, torch.LongTensor([len(char_list)])) # Obtain the next token prediction y

            last_time_step_pred = y[0, -1, :]

            next_char_id = torch.argmax(last_time_step_pred).item()

            next_char = next_char_id # Use argmax function to get the next token prediction

            if next_char == char_to_id['<eos>']:
                break

            char_list.append(next_char)

        return [id_to_char[ch_id] for ch_id in char_list]

In [ ]:
torch.manual_seed(2)

if not torch.cuda.is_available():
    raise RuntimeError("GPU not available. Please change the runtime type to GPU.")
device = torch.device('cuda')
print(f"Device set to: {device}")

model = CharRNN(vocab_size,
                embed_dim,
                hidden_dim)

model.to(device)

Device set to: cuda


CharRNN(
  (embedding): Embedding(18, 256, padding_idx=0)
  (rnn_layer1): LSTM(256, 256, batch_first=True)
  (rnn_layer2): LSTM(256, 256, batch_first=True)
  (linear): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=18, bias=True)
  )
)

In [ ]:
criterion = torch.nn.CrossEntropyLoss(ignore_index=char_to_id['<pad>'])
# Write your code here. Cross-entropy loss function. The loss function should ignore <pad>
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)# Write your code here. Use Adam or AdamW for Optimizer

# Training
1. The outer `for` loop controls the `epoch`
    1. The inner `for` loop uses `data_loader` to retrieve batches.
        1. Pass the batch to the `model` for training.
        2. Compare the predicted results `batch_pred_y` with the true labels `batch_y` using Cross Entropy to calculate the loss `loss`
        3. Use `loss.backward` to automatically compute the gradients.
        4. Use `torch.nn.utils.clip_grad_value_` to limit the gradient values between `-grad_clip` &lt; and &lt; `grad_clip`.
        5. Use `optimizer.step()` to update the model (backpropagation).
2.  After every `1000` batches, output the current loss to monitor whether it is converging.

In [ ]:
from tqdm import tqdm
from copy import deepcopy
model = model.to(device)
model.train()
i = 0
for epoch in range(1, epochs+1):
    # The process bar
    bar = tqdm(dl_train, desc=f"Train epoch {epoch}")
    for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
        # Write your code here
        # Clear the gradient
        optimizer.zero_grad()

        batch_pred_y = model(batch_x.to(device), batch_x_lens)

        # Write your code here
        # Input the prediction and ground truths to loss function
        # Back propagation
        loss = criterion(batch_pred_y.view(-1, vocab_size), batch_y.to(device).view(-1))
        loss.backward()

        torch.nn.utils.clip_grad_value_(model.parameters(), grad_clip) # gradient clipping

        # Write your code here
        # Optimize parameters in the model
        optimizer.step()

        i+=1
        if i%50==0:
            bar.set_postfix(loss = loss.item())

    print(f"Starting evaluation, model is on device: {next(model.parameters()).device}")

    # Evaluate your model
    model.eval()
    matched = 0
    total = 0
    bar_eval = tqdm(df_eval.iterrows(), desc=f"Validation epoch {epoch}")
    for _, row in bar_eval:
        batch_x = row['src']
        batch_y = row['tgt']

        #prediction = # An example of using generator: model.generator('1+1=')

        # Write your code here. Input the batch_x to the model and generate the predictions
        # Write your code here.
        # Check whether the prediction match the ground truths
        # Compute exact match (EM) on the eval dataset
        # EM = correct/total
        # 1. 使用 generator 函式生成預測的字元列表
        prediction_chars = model.generator(batch_x)

        # 2. 將字元列表轉換回字串
        prediction_str = "".join(prediction_chars)

        # 3. 從預測字串中抽取出 '=' 後面的答案
        if '=' in prediction_str:
            predicted_answer = prediction_str.split('=', 1)[1]
        else:
            predicted_answer = ""  # 如果模型沒有生成 '='，則答案視為空

        # 4. 比較預測答案與真實答案是否完全相符
        if predicted_answer == batch_y:
            matched += 1
        total += 1


model.train() # 評估結束後，將模型切換回訓練模式
print(matched/total)

Train epoch 1: 100%|██████████| 37020/37020 [07:34<00:00, 81.42it/s, loss=2.13e-8]


Starting evaluation, model is on device: cuda:0


Validation epoch 1: 1068it [11:23,  1.56it/s]


KeyboardInterrupt: 

In [ ]:
!nvidia-smi